In [5]:
import time

x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

In [1]:
def calculate_z_serial_purepython(maxiter, zs, cs):
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z*z + c
            n += 1
        output[i] = n
    return output

In [2]:
def calc_pure_python(desired_width, max_iterations):
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
        
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))
            
    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + "took", secs, "seconds")
    
    assert sum(output) == 33219980

In [6]:
calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepythontook 8.466514110565186 seconds


In [7]:
import pstats

In [8]:
p = pstats.Stats("profile.stats")

In [9]:
p.sort_stats("cumulative")

In [10]:
p.print_stats()

Tue Nov  7 21:22:38 2017    profile.stats

         36221995 function calls in 13.374 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   13.374   13.374 {built-in method builtins.exec}
        1    0.028    0.028   13.374   13.374 julia1_nopil.py:1(<module>)
        1    0.746    0.746   13.346   13.346 julia1_nopil.py:18(calc_pure_python)
        1    9.735    9.735   12.450   12.450 julia1_nopil.py:6(calculate_z_serial_purepython)
 34219980    2.715    0.000    2.715    0.000 {built-in method builtins.abs}
  2002000    0.142    0.000    0.142    0.000 {method 'append' of 'list' objects}
        1    0.007    0.007    0.007    0.007 {built-in method builtins.sum}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        2    0.000    0.000    0.000    0.000 {built-in method time.time}
     

In [11]:
p.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
{built-in method builtins.exec}                   <- 
julia1_nopil.py:1(<module>)                       <-       1    0.028   13.374  {built-in method builtins.exec}
julia1_nopil.py:18(calc_pure_python)              <-       1    0.746   13.346  julia1_nopil.py:1(<module>)
julia1_nopil.py:6(calculate_z_serial_purepython)  <-       1    9.735   12.450  julia1_nopil.py:18(calc_pure_python)
{built-in method builtins.abs}                    <- 34219980    2.715    2.715  julia1_nopil.py:6(calculate_z_serial_purepython)
{method 'append' of 'list' objects}               <- 2002000    0.142    0.142  julia1_nopil.py:18(calc_pure_python)
{built-in method builtins.sum}                    <-       1    0.007    0.007  julia1_nopil.py:18(calc_pure_python)
{built-in method builtins.print}                  <-       3    0.0

In [12]:
p.print_callees()

   Ordered by: cumulative time

Function                                          called...
                                                      ncalls  tottime  cumtime
{built-in method builtins.exec}                   ->       1    0.028   13.374  julia1_nopil.py:1(<module>)
julia1_nopil.py:1(<module>)                       ->       1    0.746   13.346  julia1_nopil.py:18(calc_pure_python)
julia1_nopil.py:18(calc_pure_python)              ->       1    9.735   12.450  julia1_nopil.py:6(calculate_z_serial_purepython)
                                                           2    0.000    0.000  {built-in method builtins.len}
                                                           3    0.000    0.000  {built-in method builtins.print}
                                                           1    0.007    0.007  {built-in method builtins.sum}
                                                           2    0.000    0.000  {built-in method time.time}
                                

In [2]:
import dis
import julia1_nopil
dis.dis(julia1_nopil.calculate_z_serial_purepython)

  7           0 LOAD_CONST               1 (0)
              2 BUILD_LIST               1
              4 LOAD_GLOBAL              0 (len)
              6 LOAD_FAST                1 (zs)
              8 CALL_FUNCTION            1
             10 BINARY_MULTIPLY
             12 STORE_FAST               3 (output)

  8          14 SETUP_LOOP              94 (to 110)
             16 LOAD_GLOBAL              1 (range)
             18 LOAD_GLOBAL              0 (len)
             20 LOAD_FAST                1 (zs)
             22 CALL_FUNCTION            1
             24 CALL_FUNCTION            1
             26 GET_ITER
        >>   28 FOR_ITER                78 (to 108)
             30 STORE_FAST               4 (i)

  9          32 LOAD_CONST               1 (0)
             34 STORE_FAST               5 (n)

 10          36 LOAD_FAST                1 (zs)
             38 LOAD_FAST                4 (i)
             40 BINARY_SUBSCR
             42 STORE_FAST               6 (z)

 11   